# Imports

In [25]:
import pandas as pd
from datetime import datetime, timedelta
import json
import time
import os
import numpy as np


In [26]:
# Data directory based on root folder
DATA_DIR = 'data'


# GET ABSOLUTE PATH
head, _ = os.path.split(os.getcwd()) # Get parent directory, from notebooks
DATA_DIR = os.path.join(head, 'data')

# DATA LOADING

In [27]:
def load_data(filename: str, prefix: str="", data_dir: os.PathLike=DATA_DIR):
    """Load data file with a specific prefix"""

    filename = f"{prefix}-{filename}" if len(prefix) else filename

    filepath = os.path.join(data_dir, filename)
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Data file not found: {filepath}. Run scraping first.")

    df = pd.read_csv(filepath)
    df.columns = df.columns.str.strip()
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # if 'date' in df.columns:
    #     df['date'] = pd.to_datetime(df['date'])

    return df

# ============================================================================
# DATA LOADING FUNCTIONS WINDSPEED AND DIRECTION
# ============================================================================

def load_all_stations(input_dir: os.PathLike =DATA_DIR, file_pattern: str=""):
    """Load data for all stations"""
    import glob
    pattern = os.path.join(input_dir, file_pattern)
    station_files = glob.glob(pattern)

    station_dfs = []
    for filepath in station_files:
        filename = os.path.basename(filepath)
        station_dfs.append(load_data(filename, data_dir=input_dir))

    return pd.concat(station_dfs).reset_index(drop=True)

def load_stations_metadata(input_dir: os.PathLike=DATA_DIR):
    """Load stations metadata"""
    filename = os.path.join(input_dir, "stations_metadata.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Metadata file not found: {filename}")

    return pd.read_csv(filename)


In [28]:
REGIONS_PREFIX = ['east', 'west', 'north', 'south', 'central']

pm25_hourly_df_temp = {region: load_data(f"pm2.5-hourly-230701-251101.csv", prefix=region, data_dir=os.path.join(DATA_DIR, "pm2.5-hourly")) for region in REGIONS_PREFIX}
pm25_hourly_df = []
for region, df in pm25_hourly_df_temp.items():
    df['region'] = region
    pm25_hourly_df.append(df)
pm25_hourly_df = pd.concat(pm25_hourly_df).reset_index(drop=True)
pm25_hourly_df['region'] = pm25_hourly_df['region'].astype('category')
pm25_hourly_df['pm25'] = pd.to_numeric(pm25_hourly_df['pm25'], errors='coerce')
pm25_hourly_df.drop(columns=['date'], axis=1, inplace=True)
del pm25_hourly_df_temp


pm25_daily_df_temp = {region: load_data(f"singapore-air-quality.csv", prefix=region, data_dir=os.path.join(DATA_DIR, "pm2.5-daily")) for region in REGIONS_PREFIX}
pm25_daily_df = []
for region, df in pm25_daily_df_temp.items():
    df['region'] = region
    pm25_daily_df.append(df)
pm25_daily_df = pd.concat(pm25_daily_df).reset_index(drop=True)
pm25_daily_df['region'] = pm25_daily_df['region'].astype('category')
pm25_daily_df.drop(columns=['pm10', 'o3', 'no2', 'so2', 'co', 'psi'], axis=1, inplace=True)
pm25_daily_df['pm25'] = pd.to_numeric(pm25_daily_df['pm25'], errors='coerce')
del pm25_daily_df_temp


wind_speed_df = load_all_stations(os.path.join(DATA_DIR, 'wind-speed'), file_pattern="*-wind-speed-hourly-*.csv")
wind_speed_df.drop(columns=['last_update'], axis=1, inplace=True)
wind_speed_df['station_name'] = wind_speed_df['station_name'].astype('category')

wind_direction_df = load_all_stations(os.path.join(DATA_DIR, 'wind-direction'), file_pattern="*-wind-direction-hourly-*.csv")
wind_direction_df.drop(columns=['last_update'], axis=1, inplace=True)
wind_direction_df['station_name'] = wind_direction_df['station_name'].astype('category')

air_temperature_df = load_all_stations(os.path.join(DATA_DIR, 'air-temperature'), file_pattern="*-air-temperature-hourly-*.csv")
air_temperature_df.drop(columns=['last_update'], axis=1, inplace=True)
air_temperature_df['station_name'] = air_temperature_df['station_name'].astype('category')


In [29]:
# PLAYGROUND AREA TO VIEW LOADED DATAFRAMES

air_temperature_df.head(6000)

,timestamp,station_name,air_temperature_avg,reading_count,latitude,longitude
0,2023-01-01 00:00:00+08:00,Old Choa Chu Kang Road,25.961667,60.0,1.37288,103.72244
1,2023-01-01 01:00:00+08:00,Old Choa Chu Kang Road,25.696667,60.0,1.37288,103.72244
2,2023-01-01 02:00:00+08:00,Old Choa Chu Kang Road,25.528333,60.0,1.37288,103.72244
3,2023-01-01 03:00:00+08:00,Old Choa Chu Kang Road,25.128333,60.0,1.37288,103.72244
4,2023-01-01 04:00:00+08:00,Old Choa Chu Kang Road,25.216667,60.0,1.37288,103.72244
...,...,...,...,...,...,...
5995,2023-09-24 03:00:00+08:00,Old Choa Chu Kang Road,26.410000,60.0,1.37288,103.72244
5996,2023-09-24 04:00:00+08:00,Old Choa Chu Kang Road,26.251667,60.0,1.37288,103.72244
5997,2023-09-24 05:00:00+08:00,Old Choa Chu Kang Road,26.160000,60.0,1.37288,103.72244
5998,2023-09-24 06:00:00+08:00,Old Choa Chu Kang Road,26.161667,60.0,1.37288,103.72244


In [30]:
pm25_hourly_df.head()

,timestamp,pm25,region
0,2023-07-01 00:00:00+08:00,13.0,east
1,2023-07-01 01:00:00+08:00,14.0,east
2,2023-07-01 02:00:00+08:00,11.0,east
3,2023-07-01 03:00:00+08:00,16.0,east
4,2023-07-01 04:00:00+08:00,7.0,east


# FEATURE ENGINEERING FOR REGRESSION

In [31]:
from typing import Callable


def regression_features_pm25_daily(df: pd.DataFrame):
    """Prepare features for time series regression modeling"""
    df = df.copy().sort_values('timestamp').reset_index(drop=True)

    # Time-based features
    df['day_of_week'] = (df['timestamp'].dt.dayofweek).astype('int8')
    df['day_of_month'] = (df['timestamp'].dt.day).astype('int8')
    df['month'] = (df['timestamp'].dt.month).astype('int8')
    df['year'] = df['timestamp'].dt.year
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

    # Lag features
    for lag in [1, 2, 3, 4, 5, 6, 7, 14, 28]:
        df[f'pm25_lag_{lag}d'] = df['pm25'].shift(lag)

    # Rolling statistics
    for window in [3, 7, 14, 28]:
        min_valid = max(1, window // 2)
        df[f'pm25_rolling_mean_{window}d'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'pm25_rolling_std_{window}d'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).std()
        df[f'pm25_rolling_min_{window}d'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).min()
        df[f'pm25_rolling_max_{window}d'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).max()

    df_clean = df.dropna()

    return df_clean


def regression_features_pm25_hourly(df: pd.DataFrame):
    """Prepare features for time series regression modeling"""
    df = df.copy().sort_values('timestamp').reset_index(drop=True)

    # Time-based features
    df['hour'] = (df['timestamp'].dt.hour).astype('int8')
    df['day_of_week'] = (df['timestamp'].dt.dayofweek).astype('int8')
    df['day_of_month'] = (df['timestamp'].dt.day).astype('int8')
    df['month'] = (df['timestamp'].dt.month).astype('int8')
    df['year'] = df['timestamp'].dt.year
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(bool)

    # Lag features
    for lag in [1, 2, 3, 6, 12, 24]:
        df[f'pm25_lag_{lag}h'] = df['pm25'].shift(lag)

    # Rolling statistics
    for window in [6, 12, 24, 72, 168]:
        min_valid = max(1, window // 2)
        df[f'pm25_rolling_mean_{window}h'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'pm25_rolling_std_{window}h'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).std()
        df[f'pm25_rolling_min_{window}h'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).min()
        df[f'pm25_rolling_max_{window}h'] = df['pm25'].shift(1).rolling(window, min_periods=min_valid).max()

    df_clean = df.dropna()

    return df_clean


def regression_features_wind_direction(df: pd.DataFrame):
    """Prepare features for time series regression modeling"""
    df = df.copy().sort_values('timestamp').reset_index(drop=True)

    # Time-based features
    df['hour'] = (df['timestamp'].dt.hour).astype('int8')
    df['day_of_week'] = (df['timestamp'].dt.dayofweek).astype('int8')
    df['day_of_month'] = (df['timestamp'].dt.day).astype('int8')
    df['month'] = (df['timestamp'].dt.month).astype('int8')
    df['year'] = df['timestamp'].dt.year
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(bool)

    # Convert direction to vector components (recommended for modeling)
    df['wind_u'] = -np.sin(np.deg2rad(df['wind_direction_avg']))
    df['wind_v'] = -np.cos(np.deg2rad(df['wind_direction_avg']))

    # Lag features (using vector components)
    for lag in [1, 2, 3, 6, 12, 24, 48, 72, 168]:
        df[f'wind_u_lag_{lag}h'] = df['wind_u'].shift(lag)
        df[f'wind_v_lag_{lag}h'] = df['wind_v'].shift(lag)
        df[f'wind_direction_lag_{lag}h'] = df['wind_direction_avg'].shift(lag)

    # Rolling statistics
    for window in [6, 12, 24, 72, 168]:
        min_valid = max(1, window // 2)
        df[f'wind_u_rolling_mean_{window}h'] = df['wind_u'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'wind_v_rolling_mean_{window}h'] = df['wind_v'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'direction_std_rolling_mean_{window}h'] = df['wind_direction_std'].shift(1).rolling(window, min_periods=min_valid).mean()

    df_clean = df.dropna()

    return df_clean


def regression_features_wind_speed(df: pd.DataFrame):
    """Prepare features for time series regression modeling"""
    df = df.copy().sort_values('timestamp').reset_index(drop=True)

    # Time-based features
    df['hour'] = (df['timestamp'].dt.hour).astype('int8')
    df['day_of_week'] = (df['timestamp'].dt.dayofweek).astype('int8')
    df['day_of_month'] = (df['timestamp'].dt.day).astype('int8')
    df['month'] = (df['timestamp'].dt.month).astype('int8')
    df['year'] = df['timestamp'].dt.year
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(bool)

    # Lag features
    for lag in [1, 2, 3, 6, 12, 24, 48, 72, 168]:
        df[f'wind_speed_lag_{lag}h'] = df['wind_speed_avg'].shift(lag)

    # Rolling statistics
    for window in [6, 12, 24, 72, 168]:
        min_valid = max(1, window // 2)
        df[f'wind_speed_rolling_mean_{window}h'] = df['wind_speed_avg'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'wind_speed_rolling_std_{window}h'] = df['wind_speed_avg'].shift(1).rolling(window, min_periods=min_valid).std()
        df[f'wind_speed_rolling_min_{window}h'] = df['wind_speed_avg'].shift(1).rolling(window, min_periods=min_valid).min()
        df[f'wind_speed_rolling_max_{window}h'] = df['wind_speed_avg'].shift(1).rolling(window, min_periods=min_valid).max()

    df_clean = df.dropna()

    return df_clean


def regression_features_air_temperature(df: pd.DataFrame):
    """Prepare features for time series regression modeling"""
    df = df.copy().sort_values('timestamp').reset_index(drop=True)

    # Time-based features
    df['hour'] = (df['timestamp'].dt.hour).astype('int8')
    df['day_of_week'] = (df['timestamp'].dt.dayofweek).astype('int8')
    df['day_of_month'] = (df['timestamp'].dt.day).astype('int8')
    df['month'] = (df['timestamp'].dt.month).astype('int8')
    df['year'] = df['timestamp'].dt.year
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(bool)

    # Lag features
    for lag in [1, 2, 3, 6, 12, 24, 48, 72, 168]:
        df[f'air_temperature_lag_{lag}h'] = df['air_temperature_avg'].shift(lag)

    # Rolling statistics
    for window in [6, 12, 24, 72, 168]:
        min_valid = max(1, window // 2)
        df[f'air_temperature_rolling_mean_{window}h'] = df['air_temperature_avg'].shift(1).rolling(window, min_periods=min_valid).mean()
        df[f'air_temperature_rolling_std_{window}h'] = df['air_temperature_avg'].shift(1).rolling(window, min_periods=min_valid).std()
        df[f'air_temperature_rolling_min_{window}h'] = df['air_temperature_avg'].shift(1).rolling(window, min_periods=min_valid).min()
        df[f'air_temperature_rolling_max_{window}h'] = df['air_temperature_avg'].shift(1).rolling(window, min_periods=min_valid).max()

    df_clean = df.dropna()

    return df_clean


def apply_func_to_groups(df: pd.DataFrame, group_col: str, func: Callable[[pd.DataFrame], pd.DataFrame]) -> pd.DataFrame:
    """Apply a function to each group in the DataFrame and combine results"""
    grouped = df.groupby(group_col)
    processed_groups = []

    for name, group in grouped:
        processed_group = func(group)
        processed_groups.append(processed_group)

    return pd.concat(processed_groups).reset_index(drop=True)

In [32]:
pm25_daily_df = apply_func_to_groups(pm25_daily_df, 'region', regression_features_pm25_daily)
pm25_hourly_df = apply_func_to_groups(pm25_hourly_df, 'region', regression_features_pm25_hourly)
wind_direction_df = apply_func_to_groups(wind_direction_df, 'station_name', regression_features_wind_direction)
wind_speed_df = apply_func_to_groups(wind_speed_df, 'station_name', regression_features_wind_speed)
air_temperature_df = apply_func_to_groups(air_temperature_df, 'station_name', regression_features_air_temperature)


/var/folders/52/d2m_fcx93ds96bs44ll4mcfr0000gn/T/ipykernel_2562/2642056818.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(group_col)
/var/folders/52/d2m_fcx93ds96bs44ll4mcfr0000gn/T/ipykernel_2562/2642056818.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(group_col)
/var/folders/52/d2m_fcx93ds96bs44ll4mcfr0000gn/T/ipykernel_2562/2642056818.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default a

In [33]:
print(pm25_daily_df.info())
# print(pm25_daily_df.dtypes)
# print(pm25_daily_df.columns)
print(pm25_hourly_df.info())
# print(pm25_hourly_df.dtypes)
# print(pm25_hourly_df.columns)
print(wind_direction_df.info())
# print(wind_direction_df.dtypes)
# print(wind_direction_df.columns)
print(wind_speed_df.info())
# print(wind_speed_df.dtypes)
# print(wind_speed_df.columns)
print(air_temperature_df.info())
# print(air_temperature_df.dtypes)
# print(air_temperature_df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20774 entries, 0 to 20773
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              20774 non-null  datetime64[ns]
 1   pm25                   20774 non-null  float64       
 2   region                 20774 non-null  category      
 3   day_of_week            20774 non-null  int8          
 4   day_of_month           20774 non-null  int8          
 5   month                  20774 non-null  int8          
 6   year                   20774 non-null  int32         
 7   is_weekend             20774 non-null  int64         
 8   pm25_lag_1d            20774 non-null  float64       
 9   pm25_lag_2d            20774 non-null  float64       
 10  pm25_lag_3d            20774 non-null  float64       
 11  pm25_lag_4d            20774 non-null  float64       
 12  pm25_lag_5d            20774 non-null  float64       
 13  p

# Data Ingestion

In [38]:
import hopsworks
project = hopsworks.login(engine="python", project="akeelaf")
fs = project.get_feature_store()

2025-11-10 13:34:15,942 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 13:34:15,945 INFO: Initializing external client
2025-11-10 13:34:15,945 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-10 13:34:16,572 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-11-10 13:34:17,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1277076


In [ ]:
%%script echo skipping --no-raise-error

# Feature Group 1: Hourly PM2.5 features
fg_pm25_hourly = fs.get_or_create_feature_group(
    name="pm25_hourly",
    description="Hourly PM2.5 features with short-term patterns",
    version=1,
    primary_key=["timestamp"],
    partition_key=["region"],
    event_time="timestamp",
)
fg_pm25_hourly.insert(pm25_hourly_df)

# Feature Group 2: Daily PM2.5 features
fg_pm25_daily = fs.get_or_create_feature_group(
    name="pm25_daily",
    description="Daily PM2.5 aggregations for long-term trends",
    version=1,
    primary_key=["timestamp"],
    partition_key=["region"],
    event_time="timestamp",
)
fg_pm25_daily.insert(pm25_daily_df)

# Feature Group 3: Wind Direction features
fg_wind_direction = fs.get_or_create_feature_group(
    name="wind_direction_hourly",
    description="Wind direction features with vector components",
    version=1,
    primary_key=["timestamp"],
    partition_key=["station_name"],
    event_time="timestamp",
)
fg_wind_direction.insert(wind_direction_df)

# Feature Group 4: Wind Speed features
fg_wind_speed = fs.get_or_create_feature_group(
    name="wind_speed_hourly",
    description="Wind speed features",
    version=1,
    primary_key=["timestamp"],
    partition_key=["station_name"],
    event_time="timestamp",
)
fg_wind_speed.insert(wind_speed_df)

# Feature Group 5: Air Temperature features
fg_air_temperature = fs.get_or_create_feature_group(
    name="air_temperature_hourly",
    description="Air temperature features",
    version=1,
    primary_key=["timestamp"],
    partition_key=["station_name"],
    event_time="timestamp",
)
fg_air_temperature.insert(air_temperature_df)



skipping


In [36]:
FORECASRT_HOURS_AHEAD = 24
# Target variables
df['target'] = df['wind_speed_avg'].shift(-FORECASRT_HOURS_AHEAD)

df['target'] = df['wind_direction_avg'].shift(-FORECASRT_HOURS_AHEAD)

df['target'] = df['pm25'].shift(-FORECASRT_HOURS_AHEAD)


KeyError: 'wind_speed_avg'